## Improvements:
* use vectorized operations raw for numpy.
* pwede mo hanapin numpy operations about divide
* isipin if euclidean or manhattan ang gagamitin
* handle pano kung lahat same value.
* dapat ang cluster values are not equal
* reduce dimensions (PCA)


In [1]:
import pandas as pd
import numpy as np
import time
#dataframe that keeps the data
df = pd.read_csv("default.csv")
#set this to the desired number of inputs
numCentroids = 4

#set this to max iterations
maxIteration = 1000

data = df.values

In [4]:
'''
Get k centroids
Check each if san pinakamalapit
then average. then thats the new centroid
loop until converges or reached limit
'''

def cluster(numCentroids, maxIteration):    
    '''
    numFeatures - number of features or columns of the data
    dataClusters - the clusters of each row of the data
    centroids - centroids and its features
    centMembCount - centtroid member count. each data row is assigned to a centroid. used for averaging
    centFeatureSum - sum of all feature values of rows that are nearest to each centroid. used for averaging 
    iteration - iteration number
    '''
    numFeatures = len(data[0])
    
    dataClusters = np.zeros(len(data))
    centroids = getInitialCentroids(data, numCentroids)
    centMembCount = np.zeros((numCentroids,1), dtype=np.int)
    centFeatureSum = np.zeros((numCentroids, numFeatures))
    iteration = 0
    while(True):
        centroidsCopy = centroids.copy() #gets a copy of the centroid

        #finds the nearest centroid
        for i, row in enumerate(data):  
            nearestCentroid = getNearestCentroid(centroids, row, numCentroids)
            dataClusters[i] = nearestCentroid #sets the centroid of the row
            centMembCount[nearestCentroid] += 1 #adds cluster member count
            for j in range(numFeatures): #this loop adds the values to centFeatureSum
                  centFeatureSum[nearestCentroid][j] += data[i][j]        
        
        #gets new centroid values
        centroids = centFeatureSum / centMembCount
        
        #checks if centroids didnt move or program reached maximum Iteration
        if(np.array_equal(centroidsCopy, centroids )):
            print("Data has converged")
            break
        elif  iteration > maxIteration:
            print("Data reached max iteration")
            break
        else:
            #reset values
            centMembCount[:] = 0
            centFeatureSum[:] = 0

            iteration += 1
    
    print("Iteration", iteration)
    print("dataClusters", dataClusters)
    print("centroids", centroids)
    print("data", data)
    print("centMembCount", centMembCount)
    
def getNearestCentroid(centroids, row, numCentroids):
    dist = np.linalg.norm(centroids[0]-row) #get distance
    nearestC = 0
    
    for i in range(1, numCentroids):
        newDist = np.linalg.norm(centroids[i] - row)
        if(newDist < dist):
            dist = newDist
            nearestC = i
    
    return nearestC 
 
def getInitialCentroids(data, numCentroids):
    return data[:numCentroids] #returns the first n number of rows as centroids

start_time = time.time()
cluster(numCentroids, maxIteration)
print("--- %s seconds ---" % (time.time() - start_time))

Data has converged
Iteration 17
dataClusters [ 0.  1.  1. ...,  2.  0.  0.]
centroids [[  1.73488199e-01   3.93330943e-01   6.06669057e-01   4.50340624e-01
    4.01936178e-01   1.44137684e-01   3.58551452e-03   4.77949086e-01
    5.13445679e-01   8.60523485e-03   2.56667202e-01   3.11939763e-02
    5.65794191e-01   6.38221585e-02   1.74614557e-01   1.28719971e-01
    2.50986016e-02   4.66116888e-03   1.43420581e-03   1.43420581e-03
    1.07565436e-03   2.15130871e-03   0.00000000e+00   5.05557548e-02
    6.15274292e-01   4.76873431e-02   1.79275726e-03   2.41663679e-01
    2.61742560e-02   9.32233776e-03   3.58551452e-03   1.79275726e-03
    2.15130871e-03   0.00000000e+00   0.00000000e+00   4.41018286e-02
    6.33201864e-01   3.97992112e-02   3.58551452e-04   2.43814987e-01
    2.00788813e-02   8.24668340e-03   2.50986016e-03   2.86841162e-03
    4.30261743e-03   7.17102904e-04   0.00000000e+00   3.69307996e-02
    6.00573682e-01   1.01828612e-01   3.58551452e-04   2.21943349e-01
    